In [1]:
# Import the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import keras
from keras.models import Sequential
from keras.layers import Dense

import matplotlib.pyplot as plt

In [2]:
# Download data and split training & test sets 
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Split columns into Predictors and Target
concrete_data_columns = concrete_data.columns

P = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
T = concrete_data['Strength'] # Strength column

In [4]:
# Mormalise the Predictors
P_norm = (P - P.mean()) / P.std()
P_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [5]:
# number of predictors
n_cols = P_norm.shape[1] 

In [6]:
# Define the regression model with one hidden layer
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Build the model
model = regression_model()

In [7]:
# We create a 50-cycle training & evaluation loop. We would train the model on a 50-epoch fit -per cycle- and evaluate using MSE per cycle. We record each cycle MSE into a list.

score_mse_list = [] # Create a list to record the MSEs per cycle  
cycle  = 0 # Initialise the cycles

for looping in range(50): # Loop 50 times
    
    # Split the Predictors & Target data, holding 30% for testing
    P_norm_train, P_norm_test, T_train, T_test = train_test_split(P_norm, T,
                                                    test_size = 0.3,
                                                    random_state = 1)
    #Initialise the cycle MSE
    score_mse = 0
    
    # Fit the model 
    model.fit( P_norm_train, T_train, epochs=50, verbose=2)
    
    # Generate predicted test target values
    T_test_hat = model.predict(P_norm_test)
    
    # Cycle evaluation using scikit-learn mean_squared_error
    score_mse = mean_squared_error(T_test, T_test_hat) 
    
    # Append the loop MSE on the list
    score_mse_list.append(score_mse)

Epoch 1/50
23/23 - 0s - loss: 1601.1332 - 344ms/epoch - 15ms/step
Epoch 2/50
23/23 - 0s - loss: 1584.9912 - 18ms/epoch - 783us/step
Epoch 3/50
23/23 - 0s - loss: 1569.8502 - 16ms/epoch - 696us/step
Epoch 4/50
23/23 - 0s - loss: 1555.2491 - 16ms/epoch - 696us/step
Epoch 5/50
23/23 - 0s - loss: 1541.6556 - 17ms/epoch - 739us/step
Epoch 6/50
23/23 - 0s - loss: 1528.5756 - 18ms/epoch - 783us/step
Epoch 7/50
23/23 - 0s - loss: 1515.9220 - 17ms/epoch - 739us/step
Epoch 8/50
23/23 - 0s - loss: 1503.7301 - 17ms/epoch - 739us/step
Epoch 9/50
23/23 - 0s - loss: 1491.6613 - 17ms/epoch - 739us/step
Epoch 10/50
23/23 - 0s - loss: 1479.6725 - 16ms/epoch - 696us/step
Epoch 11/50
23/23 - 0s - loss: 1467.7710 - 16ms/epoch - 696us/step
Epoch 12/50
23/23 - 0s - loss: 1455.5884 - 16ms/epoch - 696us/step
Epoch 13/50
23/23 - 0s - loss: 1443.2854 - 16ms/epoch - 696us/step
Epoch 14/50
23/23 - 0s - loss: 1430.4927 - 18ms/epoch - 783us/step
Epoch 15/50
23/23 - 0s - loss: 1417.2186 - 16ms/epoch - 696us/step
Epoc

In [8]:
# Printing the list of MSEs and verify the its length
print(score_mse_list)
len(score_mse_list)

[625.3681286488849, 241.93700623105767, 162.9244539109493, 110.70829867196369, 70.55065838351526, 58.33286624610423, 54.31458096535578, 51.837232731270475, 50.24773990320141, 48.579220773023614, 47.53398174708746, 46.86275668747568, 46.45348368677224, 45.817392489428656, 45.37727742331427, 45.281438343885604, 45.300882711611784, 45.19145315620583, 44.95868289267803, 44.695142889134374, 44.43763855419663, 44.07234906211204, 43.86182052132082, 43.8787485496028, 43.79894626152854, 43.345375660432985, 43.169617320815256, 42.440727640985955, 42.389426223111656, 42.04215881646913, 42.157684038532736, 41.952635761631214, 41.678682622794234, 41.54909392484305, 41.60653859795657, 41.336003042807356, 41.382642608527675, 41.43728896296575, 41.45798882695688, 41.230334076994346, 41.211994006733, 40.929728970069036, 41.01637957805864, 41.01056415817743, 40.75735100475474, 40.60638595280323, 40.367263015823355, 40.40337462024516, 40.487600235133776, 40.23596209005015]


50

In [9]:
# Calculate and report the mean & std of the MSEs from the list
print('The training values mean of the 50-cycle MSEs is :', np.mean(score_mse_list))
print('The training values standard deviation of the 50-cycle MSEs is :', np.std(score_mse_list))

The training values mean of the 50-cycle MSEs is : 63.77049966398708
The training values standard deviation of the 50-cycle MSEs is : 86.89912618512398


## The mean of the MSEs has sightly increased, while the standard deviation has radically increased after normalization.